Before running this notebook, ensure you have installed spark-cloudant 1.6.4 by running the notebook: **Install spark-cloudant 1.6.4 lib**

In [15]:
! python -c 'import cloudant' || pip install cloudant --user

In [16]:
# utility method for timestamps
import time
def ts():
    return time.strftime("%Y-%m-%d %H:%M:%S %Z")

In [17]:
# utility method for logging
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger("CloudantRecommender")

def info(*args):
    
    # sends output to notebook
    print(args)
    
    # sends output to kernel log file
    LOGGER.info(args)
    
def error(*args):
    
    # sends output to notebook
    print(args)
    
    # sends output to kernel log file
    LOGGER.error(args)

In [18]:
# utility class for holding cloudant connection details
import json

def set_attr_if_exists(obj, data, k):
    try:
        setattr(obj, k, data[k])
    except AttributeError:
        pass

class CloudantConfig:
    def __init__(self, database, json_file=None, host=None, username=None, password=None):
       
        self.database = database
        self.host = None
        self.username = None
        self.password = None

        with open(json_file) as data_file:    
            data = json.load(data_file)
            
            set_attr_if_exists(self, data, 'host')
            set_attr_if_exists(self, data, 'username')
            set_attr_if_exists(self, data, 'password')
        
        # override json attributes if provided
        if host:     self.host = host
        if username: self.username = username
        if password: self.password = password

In [19]:
sourceDB = CloudantConfig(
                    json_file='cloudant_credentials.json', 
                    database="ratingdb"
                    )

print(sourceDB.host)

9aefd1f0-d288-4666-a12f-abd93ee724fc-bluemix.cloudant.com


In [20]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

import json

# we use the cloudant python library to save the recommendations
from cloudant.client import Cloudant
from cloudant.adapters import Replay429Adapter

class CloudantMovieRecommender:
    
    def __init__(self, sc):
        self.sc = sc
    
    def train(self, sourceDB):
                      
        info("Starting load from Cloudant: ", ts())

        dfReader = sqlContext.read.format("com.cloudant.spark")
        dfReader.option("cloudant.host", sourceDB.host)
        
        if sourceDB.username:
            dfReader.option("cloudant.username", sourceDB.username)
            
        if sourceDB.password:
            dfReader.option("cloudant.password", sourceDB.password)
            
        df = dfReader.load(sourceDB.database).cache()

        info("Finished load from Cloudant: ", ts())
        info("Found", df.count(), "records in Cloudant")
        
        # convert cloudant docs into Rating objects
        def make_rating(row):
            (user_id, prod_id) = row[0].split('/')
            user_id = int(user_id.replace('user_', ''))
            prod_id = int(prod_id.replace('movie_', ''))

            rating = float(row[2])
            return Rating(user_id, prod_id, rating)
        
        ratings = df.map(make_rating)

        rank = 50
        numIterations = 20
        lambdaParam = 0.1

        info("Starting train model: ", ts())
        self.model = ALS.train(ratings, rank, numIterations, lambdaParam)
        info("Finished train model: ", ts())
        
    def get_top_recommendations(self):
        info("Starting __get_top_recommendations: ", ts())
        df = self.model.recommendProductsForUsers(10).toDF()
        df.cache()
        info("Finished __get_top_recommendations: ", ts())
        return df
    
    def del_old_recommendationdbs(self, cloudant_client, db_name_prefix):
        dbs_to_del = cloudant_client.all_dbs()

        # only delete dbs we are using for recommendations
        dbs_to_del = [db for db in dbs_to_del if db.startswith(db_name_prefix + '_') ]

        # ensure the list is in timestamp order
        dbs_to_del.sort()

        # keeping the last 5 dbs and delete the rest
        for db in dbs_to_del[:-5]:
            cloudant_client.delete_database(db)
            info("Deleted old recommendations db", db)
            
    def update_meta_document(self, cloudant_client, meta_db_name, latest_db_name):
        
        meta_db = cloudant_client[meta_db_name]
        
        try:
            # update doc if exists
            meta_doc = meta_db['recommendation_metadata']
            meta_doc['latest_db'] = latest_db_name
            meta_doc['timestamp'] = ts()
            meta_doc.save()
            info("Updated recommendationdb metadata record with latest_db", latest_db_name)
        except KeyError:
            # create a new doc
            data = {
                '_id': 'recommendation_metadata',
                'latest_db': latest_db_name,
                'timestamp': ts()
                }
            meta_doc = meta_db.create_document(data)
            if meta_doc.exists():
                info("Saved recommendationdb metadata record", str(data))
    
    def create_recommendationdb(self, cloudant_client):
        # create a database for recommendations
        import time
        db_name = destDB.database + '_' + str(int(time.time()))
        
        db = cloudant_client.create_database(db_name)
        info("Created new recommendations db", db_name)
        return db
        
    def save_recommendations(self, destDB):
        df = movieRecommender.get_top_recommendations()
        
        cloudant_client = Cloudant(
                                destDB.username,
                                destDB.password,
                                account=destDB.username, 
                                adapter=Replay429Adapter(retries=10)
                                )
        cloudant_client.connect()
        self.del_old_recommendationdbs(cloudant_client, destDB.database)
        recommendations_db = self.create_recommendationdb(cloudant_client)

        # reformat data for saving
        docs = df.map(lambda x: {'_id':str(x[0]), 'recommendations':x[1]}).collect()
        
        # we could hit cloudant resource limits if trying to save entire doc
        # so we save it in smaller sized chunks
        
        for i in range(0, len(docs), 100):
            chunk = docs[i:i + 100]
            recommendations_db.bulk_docs(chunk) # TODO check for errors saving the chunk
            info("Saved recommendations chunk", i, ts())
        
        self.update_meta_document(cloudant_client, destDB.database, recommendations_db.database_name)
        
        info("Saved recommendations to: ", recommendations_db.database_name, ts())

        cloudant_client.disconnect()

Add your cloudant credentials below, change notebook format to 'Code' and run the cell to save your credentials

In [21]:
sourceDB = CloudantConfig(
                    json_file='cloudant_credentials.json', 
                    database="ratingdb"
                    )

destDB = CloudantConfig(
                    json_file='cloudant_credentials.json', 
                    database="recommendationdb", 
                    )

import traceback
try:
    movieRecommender = CloudantMovieRecommender(sc)
    movieRecommender.train(sourceDB)
    movieRecommender.save_recommendations(destDB)
except Exception as e:
    error(str(e), traceback.format_exc(), ts())
    raise e

('Starting load from Cloudant: ', '2017-01-05 15:40:13 CST')
('Finished load from Cloudant: ', '2017-01-05 15:41:16 CST')
('Found', 1000009, 'records in Cloudant')
('Starting train model: ', '2017-01-05 15:42:47 CST')
('Finished train model: ', '2017-01-05 15:43:24 CST')
('Starting __get_top_recommendations: ', '2017-01-05 15:43:24 CST')
('Finished __get_top_recommendations: ', '2017-01-05 15:43:28 CST')
('Created new recommendations db', 'recommendationdb_1483652608')
('Saved recommendations chunk', 0, '2017-01-05 15:43:30 CST')
('Saved recommendations chunk', 100, '2017-01-05 15:43:30 CST')
('Saved recommendations chunk', 200, '2017-01-05 15:43:31 CST')
('Saved recommendations chunk', 300, '2017-01-05 15:43:31 CST')
('Saved recommendations chunk', 400, '2017-01-05 15:43:31 CST')
('Saved recommendations chunk', 500, '2017-01-05 15:43:31 CST')
('Saved recommendations chunk', 600, '2017-01-05 15:43:31 CST')
('Saved recommendations chunk', 700, '2017-01-05 15:43:31 CST')
('Saved recommen

## For debugging issues

In [22]:
# dump the latest kernel log
! cat $(ls -1 $HOME/logs/notebook/*pyspark* | sort -r | head -1)

========================== 20170105_212445 ==========================
/usr/local/src/bluemix_jupyter_bundle.v31/provision/pyspark_kernel_wrapper.sh /gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/notebook/jupyter-rt/kernel-ca960e98-2971-44f9-9e2c-25f125822604.json spark160master
no extra config
load default config from : /gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/notebook/spark-config/spark160master
-------- Environment for PySpark --------
APP_ENV_BM_DOMAIN=ng.bluemix.net
APP_ENV_CDSX_NOTEBOOKS_API=cdsx-notebooks-api.ng.bluemix.net
APP_ENV_ENVIRONMENT=prod
APP_ENV_IBM_ONLY_AUTH=false
APP_ENV_JUPYTER_TENANTS_API=cdsx-tenants-api.ng.bluemix.net
APP_ENV_NOTEBOOKS_JOB_MANAGER=cdsx-notebooks-job-manager.ng.bluemix.net
ATLAS_VERSION=3.10.2
_=/bin/printenv
BLUEMIX_RES_PLAN=s
BRUNEL_CONFIG=locjavascript=/data/jupyter2/23fd2775-6db5-4d40-b15a-8ea34840daaa/nbextensions/brunel_ext
CC_DISABLE_BIG_BUFFER_API=true
CDSX_APP_ENV_NOTEBOOKS_API_URL=https://cdsx-notebooks-api.ng.bluemix.net/v1/no

In [23]:
# look for our log output in the latest kernel log file
! grep 'CloudantRecommender' $(ls -1 $HOME/logs/notebook/*pyspark* | sort -r | head -1)

17/01/05 15:25:00 INFO CloudantRecommender: [Starting load from Cloudant: , 2017-01-05 15:25:00 CST]
17/01/05 15:32:49 INFO CloudantRecommender: [Finished load from Cloudant: , 2017-01-05 15:32:49 CST]
17/01/05 15:34:01 INFO CloudantRecommender: [Found, 1000009, records in Cloudant]
17/01/05 15:34:01 INFO CloudantRecommender: [Starting train model: , 2017-01-05 15:34:01 CST]
17/01/05 15:34:52 INFO CloudantRecommender: [Finished train model: , 2017-01-05 15:34:52 CST]
17/01/05 15:34:52 INFO CloudantRecommender: [Starting __get_top_recommendations: , 2017-01-05 15:34:52 CST]
17/01/05 15:35:21 INFO CloudantRecommender: [Finished __get_top_recommendations: , 2017-01-05 15:35:21 CST]
17/01/05 15:35:21 INFO CloudantRecommender: [Deleted old recommendations db, recommendationdb_1483650939]
17/01/05 15:35:21 INFO CloudantRecommender: [Created new recommendations db, recommendationdb_1483652121]
17/01/05 15:35:23 INFO CloudantRecommender: [Saved recommendations chunk, 0, 2017-01-05 15:35:23 CST

In [24]:
# look for our log output in all kernel log files
! grep 'CloudantRecommender' $HOME/logs/notebook/*pyspark* 

/gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/logs/notebook/kernel-pyspark-20170105_164844.log:17/01/05 10:49:08 INFO CloudantRecommender: [Starting load from Cloudant: , 2017-01-05 10:49:08]
/gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/logs/notebook/kernel-pyspark-20170105_164844.log:17/01/05 10:53:21 INFO CloudantRecommender: [Finished load from Cloudant: , 2017-01-05 10:53:21]
/gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/logs/notebook/kernel-pyspark-20170105_164844.log:17/01/05 10:56:03 INFO CloudantRecommender: [Found, 1000000, records in Cloudant]
/gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/logs/notebook/kernel-pyspark-20170105_164844.log:17/01/05 10:56:03 INFO CloudantRecommender: [Starting train model: , 2017-01-05 10:56:03]
/gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/logs/notebook/kernel-pyspark-20170105_164844.log:17/01/05 10:56:34 INFO CloudantRecommender: [Finished train model: , 2017-01-05 10:56:34]
/gpfs/fs01/user/s15a-8ea34840daaa3e-39ca506ba762/logs/